In [2]:
# Basic math operations

import math
import random

def matmul(A, B):
    result = []
    for i in range(len(A)):
        row = []
        for j in range(len(B[0])):
            sum = 0
            for k in range(len(A[0])):
                sum += A[i][k] * B[k][j]
            row.append(sum)
        result.append(row)
    return result

def add_vectors(a, b):
    return [[a[i][0] + b[i][0]] for i in range(len(a))]

def subtract_vectors(a, b):
    return [[a[i][0] - b[i][0]] for i in range(len(a))]

def tanh(x):
    return [[math.tanh(x[i][0])] for i in range(len(x))]

def tanh_derivative(x):
    return [[1 - math.tanh(x[i][0])**2] for i in range(len(x))]

def transpose(A):
    return [[A[j][i] for j in range(len(A))] for i in range(len(A[0]))]


In [3]:
# Initialize RNN parameters

def init_rnn(input_size, hidden_size, output_size):
    def random_matrix(rows, cols):
        return [[random.uniform(-0.1, 0.1) for _ in range(cols)] for _ in range(rows)]
    
    return {
        'W_xh': random_matrix(hidden_size, input_size),
        'W_hh': random_matrix(hidden_size, hidden_size),
        'W_hy': random_matrix(output_size, hidden_size),
        'b_h': [[0] for _ in range(hidden_size)],
        'b_y': [[0] for _ in range(output_size)]
    }


In [4]:
# Forward pass through the RNN

def rnn_forward(inputs, h_prev, params):
    W_xh, W_hh, W_hy, b_h, b_y = params.values()
    h, y = {}, {}
    h[-1] = h_prev
    for t in range(len(inputs)):
        temp1 = matmul(W_xh, inputs[t])
        temp2 = matmul(W_hh, h[t-1])
        h[t] = tanh(add_vectors(add_vectors(temp1, temp2), b_h))
        y[t] = add_vectors(matmul(W_hy, h[t]), b_y)
    return y, h


In [5]:
# Backward pass (Backpropagation Through Time - BPTT)

def rnn_backward(inputs, targets, outputs, h, params):
    W_xh, W_hh, W_hy, b_h, b_y = params.values()
    dW_xh = [[0 for _ in range(len(W_xh[0]))] for _ in range(len(W_xh))]
    dW_hh = [[0 for _ in range(len(W_hh[0]))] for _ in range(len(W_hh))]
    dW_hy = [[0 for _ in range(len(W_hy[0]))] for _ in range(len(W_hy))]
    db_h = [[0] for _ in range(len(b_h))]
    db_y = [[0] for _ in range(len(b_y))]
    
    dh_next = [[0] for _ in range(len(h[0]))]
    
    for t in reversed(range(len(inputs))):
        dy = subtract_vectors(outputs[t], targets[t])
        dW_hy_t = matmul(dy, transpose(h[t]))
        dW_hy = [[dW_hy[i][j] + dW_hy_t[i][j] for j in range(len(dW_hy[0]))] for i in range(len(dW_hy))]
        db_y = [[db_y[i][0] + dy[i][0]] for i in range(len(db_y))]

        dh = matmul(transpose(W_hy), dy)
        dh = [[dh[i][0] + dh_next[i][0]] for i in range(len(dh))]

        dh_raw = [[dh[i][0] * tanh_derivative(h[t])[i][0]] for i in range(len(dh))]
        db_h = [[db_h[i][0] + dh_raw[i][0]] for i in range(len(db_h))]

        dW_xh_t = matmul(dh_raw, transpose(inputs[t]))
        dW_xh = [[dW_xh[i][j] + dW_xh_t[i][j] for j in range(len(dW_xh[0]))] for i in range(len(dW_xh))]
        
        dW_hh_t = matmul(dh_raw, transpose(h[t-1]))
        dW_hh = [[dW_hh[i][j] + dW_hh_t[i][j] for j in range(len(dW_hh[0]))] for i in range(len(dW_hh))]

        dh_next = matmul(transpose(W_hh), dh_raw)
        
    gradients = {
        'W_xh': dW_xh,
        'W_hh': dW_hh,
        'W_hy': dW_hy,
        'b_h': db_h,
        'b_y': db_y
    }
    return gradients


In [6]:
# Testing the RNN (Example)

input_size = 2
hidden_size = 3
output_size = 1

params = init_rnn(input_size, hidden_size, output_size)

# Create random inputs and targets
inputs = [[[random.uniform(-1,1)] for _ in range(input_size)] for _ in range(3)]
targets = [[[random.uniform(-1,1)] for _ in range(output_size)] for _ in range(3)]
h_prev = [[0] for _ in range(hidden_size)]

# Forward
outputs, h = rnn_forward(inputs, h_prev, params)

# Backward
grads = rnn_backward(inputs, targets, outputs, h, params)

print("Forward outputs:")
for t in outputs:
    print(f"Time {t}: {outputs[t]}")

print("\nGradients:")
for k in grads:
    print(f"{k}: {grads[k]}")


Forward outputs:
Time 0: [[0.007916150915552318]]
Time 1: [[-0.005647810960905676]]
Time 2: [[0.003456432113094685]]

Gradients:
W_xh: [[-0.048419726362532986, -0.0038669584713361942], [-0.06011726870988073, -0.004451161998164945], [0.06766197081306215, 0.005261962973507856]]
W_hh: [[0.003748347216056887, -0.00016335025135174474, -0.0005416075090800145], [0.004058661693224363, 0.0001663970911678557, -0.0012000327179545855], [-0.0046283784969281755, -0.00014664853069729314, 0.0012914315256594342]]
W_hy: [[0.0832498713861981, -0.001473936067610624, -0.021962048479855563]]
b_h: [[0.08848458838701315], [0.09165346031325428], [-0.10530840995818846]]
b_y: [[-1.204927069454751]]


# Name: Nora Mohamed Ebrahim
# ID: 4221310
# Subgroup: A5 - AI